In [3]:
# Generic setup block

import os, sys
import pprint
from pathlib import Path

# Set paths

project_root = Path(os.getcwd()).parent
src_path     = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))  # so `import pyetm` works

# Hide the trackback for now

ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

# Import pyETM

from pyetm.config.settings import get_settings
from pyetm.models import Scenario

# Check if our API is ready!

print("Using ETM API at    ",       get_settings().base_url)
print("Token loaded?       ",       bool(get_settings().etm_api_token))

Using ETM API at     http://localhost:3000/api/v3
Token loaded?        True


In [4]:
from pyetm.clients import BaseClient
from pyetm.services.scenario_runners.fetch_carrier_curves import DownloadCarrierCurveRunner
import pandas as pd


scenario = Scenario.load(2690288)
client = BaseClient()

print(f"Curves from scenario with session id: {scenario.id}")
curve_name = "electricity_price"
result = DownloadCarrierCurveRunner.run(client, scenario, curve_name)
df = pd.read_csv(result.data)
print(f"Shape: {df.shape}")
print(df)

Curves from scenario with session id: 2690288
Shape: (8760, 2)
                  Time  Price (Euros)
0     2050-01-01 00:00           26.0
1     2050-01-01 01:00           26.0
2     2050-01-01 02:00           25.0
3     2050-01-01 03:00           25.0
4     2050-01-01 04:00           25.0
...                ...            ...
8755  2050-12-31 19:00           25.0
8756  2050-12-31 20:00           25.0
8757  2050-12-31 21:00           25.0
8758  2050-12-31 22:00           25.0
8759  2050-12-31 23:00           25.0

[8760 rows x 2 columns]


In [6]:
# Test fetch all curves

from pyetm.services.scenario_runners.fetch_carrier_curves import FetchAllCarrierCurvesRunner

curves = FetchAllCarrierCurvesRunner.run(client, scenario)

print(f"Downloaded {len(curves.data) if curves.data else 0} curves")
if curves.errors:
    print(f"Warnings: {curves.errors}")


Downloaded 10 curves


Queries can be added on-the-go. When the packer is exported to Excel, all requested queries will show up.

## Curves to pandas series

@Louis, I moved this from the excel one. It needs some cleaning up!

In [ ]:
# Custom curves
if scenario.custom_curves.is_attached("interconnector_2_price"):
    print(
        scenario.custom_curve_series("interconnector_2_price")
    )

0       26.00
1       26.00
2       14.48
3        0.01
4        0.02
        ...  
8755     0.01
8756     0.01
8757     0.01
8758     0.02
8759     0.00
Name: interconnector_2_price, Length: 8760, dtype: float64


In [ ]:
from pyetm.clients.base_client import BaseClient


curve = scenario.carrier_curves._find("electricity_price")
print(f"Curve found: {curve is not None}")

if curve:
    # Test direct curve retrieval
    direct_result = curve.retrieve(BaseClient(), scenario)
    print(f"Direct retrieve: {type(direct_result)}")

    # Test via get_contents
    contents_result = scenario.carrier_curves.get_contents(scenario, "electricity_price")
    print(f"get_contents: {type(contents_result)}")

    # Test via scenario method
    scenario_result = scenario.carrier_curve_series("electricity_price")
    print(f"scenario method: {type(scenario_result)}")


Curve found: True
Direct retrieve: <class 'pandas.core.frame.DataFrame'>
get_contents: <class 'pandas.core.frame.DataFrame'>
scenario method: <class 'pandas.core.frame.DataFrame'>


In [ ]:
print(f"Available carrier curves: {list(scenario.carrier_curves.attached_keys())}")
result = scenario.carrier_curve_series("electricity_price")
print(f"Result: {type(result)}")
if result is not None:
    print(f"Shape: {result.shape}")
    print(f"First 5 values: {result.head()}")

Available carrier curves: ['merit_order', 'electricity_price', 'heat_network', 'agriculture_heat', 'household_heat', 'buildings_heat', 'hydrogen', 'network_gas', 'residual_load', 'hydrogen_integral_cost']
Result: <class 'pandas.core.frame.DataFrame'>
Shape: (8760, 1)
First 5 values:    Price (Euros)
0           26.0
1           26.0
2           25.0
3           25.0
4           25.0


In [ ]:
curve_count = 0
failed_curves = []

for curve_name in scenario.carrier_curves.attached_keys():
    print(f"Fetching {curve_name}...", end=" ")

    try:
        # This will fetch and cache the curves
        result = scenario.carrier_curve_series(curve_name)

        if result is not None:
            print(f"({result.shape[0]} points)")
            curve_count += 1
        else:
            print("No data")
            failed_curves.append(curve_name)

    except Exception as e:
        print(f"Error: {e}")
        failed_curves.append(curve_name)

print(f"   Successfully fetched: {curve_count} curves")
print(f"   Failed: {len(failed_curves)} curves")

if failed_curves:
    print(f"   Failed curves: {failed_curves}")


Fetching merit_order... 

KeyboardInterrupt: 